In [1]:
from skshapes.morphing import ElasticMetric, RigidMotion, ElasticMetric2
from skshapes.loss import OptimalTransportLoss, LandmarkLoss, NearestNeighborsLoss
from skshapes.data import read
from skshapes.tasks import DistanceMatrix, Registration, Registration2
import torch

In [3]:
## Rigid alignment (with landmarks)

from skshapes.data import read
from skshapes.tasks import Registration
from skshapes.morphing import RigidMotion
from skshapes.loss import LandmarkLoss
from skshapes.optimization import LBFGS
from skshapes.data import read

import torch
import os

targetfolder = "data/SCAPE_low_resolution_aligned"
datafolder = "data/SCAPE_low_resolution"

files = os.listdir(datafolder)
reference = read(datafolder + "/" + files[0])
rest = [read(datafolder + "/" + file) for file in files[1:]]

N = reference.points.shape[0]
reference.landmarks = torch.arange(N, dtype=torch.int64)
reference.to_pyvista().save(targetfolder + "/" + files[0])

for i, mesh in enumerate(rest):
    print("Aligning " + files[i + 1] + " to " + files[0] + "...")
    mesh.landmarks = torch.arange(N, dtype=torch.int64)

    r = Registration(
        model=RigidMotion(),
        loss=LandmarkLoss(p=2),
        optimizer=LBFGS(),
        verbose=0,
        n_iter=2,
        device="cpu",
    )

    morphed = r.fit_transform(source=mesh, target=reference)
    morphed.to_pyvista().save(targetfolder + "/" + files[i + 1])

Aligning mesh005.ply to mesh047.ply...
Aligning mesh001.ply to mesh047.ply...
Aligning mesh042.ply to mesh047.ply...
Aligning mesh027.ply to mesh047.ply...
Aligning mesh003.ply to mesh047.ply...
Aligning mesh023.ply to mesh047.ply...
Aligning mesh026.ply to mesh047.ply...
Aligning mesh022.ply to mesh047.ply...
Aligning mesh028.ply to mesh047.ply...
Aligning mesh033.ply to mesh047.ply...
Aligning mesh021.ply to mesh047.ply...
Aligning mesh044.ply to mesh047.ply...
Aligning mesh004.ply to mesh047.ply...
Aligning mesh024.ply to mesh047.ply...
Aligning mesh070.ply to mesh047.ply...
Aligning mesh029.ply to mesh047.ply...
Aligning mesh034.ply to mesh047.ply...
Aligning mesh011.ply to mesh047.ply...
Aligning mesh048.ply to mesh047.ply...
Aligning mesh012.ply to mesh047.ply...
Aligning mesh068.ply to mesh047.ply...
Aligning mesh052.ply to mesh047.ply...
Aligning mesh000.ply to mesh047.ply...
Aligning mesh065.ply to mesh047.ply...
Aligning mesh018.ply to mesh047.ply...
Aligning mesh032.ply to m

In [1]:
##### ICP

rotation = 1

from skshapes.data import read, PolyData
from skshapes.loss import NearestNeighborsLoss
from skshapes.morphing import RigidMotion
from skshapes.tasks import Registration
from skshapes.optimization import LBFGS

import torch

# Load a mesh and apply a rotation
mesh = read("data/SCAPE_low_resolution/mesh000.ply")
r = RigidMotion()
parameter = torch.Tensor([[rotation, rotation, rotation], [0, 0, 0]])
newpoints = r.fit(source=mesh).morph(parameter=parameter)
mesh.to_pyvista().save("source.vtk")
newmesh = PolyData(points=newpoints, triangles=mesh.triangles)
newmesh.to_pyvista().save("target.vtk")


source = read("source.vtk")
target = read("target.vtk")

source.landmarks = torch.arange(source.points.shape[0], dtype=torch.int64)
target.landmarks = torch.arange(target.points.shape[0], dtype=torch.int64)

r = Registration(
    model=RigidMotion(),
    loss=NearestNeighborsLoss(),
    optimizer=LBFGS(),
    verbose=1,
    n_iter=5,
    device="cpu",
)

morphed = r.fit_transform(source=source, target=target)
morphed.to_pyvista().save("output.vtk")

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : 

In [3]:
# Elastic registration (with landmarks)

from skshapes.data import read
from skshapes.loss import LandmarkLoss, LandmarkLoss2
from skshapes.morphing import ElasticMetric, ElasticMetric2
from skshapes.tasks import Registration, Registration2
from skshapes.optimization import LBFGS

import torch
import os

datafolder = "data/SCAPE_low_resolution_aligned"

source = read(datafolder + "/" + "mesh001.ply")
target = read(datafolder + "/" + "mesh041.ply")

source.landmarks = torch.arange(source.points.shape[0], dtype=torch.int64)
target.landmarks = torch.arange(target.points.shape[0], dtype=torch.int64)

from time import time

start = time()
r = Registration(
    model=ElasticMetric(n_steps=1),
    loss=LandmarkLoss(),
    optimizer=LBFGS(),
    verbose=1,
    n_iter=5,
    regularization=100,
    device="cpu",
)
newshape = r.fit_transform(source=source, target=target)
print(time() - start)

start = time()
r = Registration2(
    model=ElasticMetric2(n_steps=1),
    loss=LandmarkLoss2(),
    optimizer=LBFGS(),
    verbose=1,
    n_iter=5,
    regularization=100,
    device="cpu",
)
newshape = r.fit_transform(source=source, target=target)
print(time() - start)

Loss value at iteration 0 : 5.045347213745117
Loss value at iteration 1 : 2.326136827468872
Loss value at iteration 2 : 2.325077533721924
Loss value at iteration 3 : 2.325077533721924
Loss value at iteration 4 : 2.325077533721924
0.10534405708312988
Loss value at iteration 0 : 5.045347213745117
Loss value at iteration 1 : 2.326136827468872
Loss value at iteration 2 : 2.3250770568847656
Loss value at iteration 3 : 2.3250770568847656
Loss value at iteration 4 : 2.3250770568847656
0.17705011367797852


In [1]:
from skshapes.preprocessing import (
    LandmarkSetter,
    Pipeline,
    Decimation,
    AffineTransformation,
)
from skshapes.data import read
import os

# Read the shapes
datafolder = "/home/GitHub/scikit-shapes-draft/data/SCAPE/decimated/"
shapes = [read(datafolder + "/" + f"aligned_mesh{i:03d}.ply") for i in range(4)]

# Create a pipeline of preprocessing steps
preprocessing = Pipeline(
    steps=[
        LandmarkSetter(landmarks=[[0, 1], [2, 3], [4, 5], [6, 70]], by_indices=True),
        Decimation(target_reduction=0.95),
        AffineTransformation(matrix=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    ]
)

# Apply the preprocessing pipeline to the shapes and get the dataset
dataset = preprocessing.fit_transform(shapes=shapes)

for i in range(len(dataset)):
    print("Size of shape {} : {}".format(i, dataset[i].points.shape))
    print("Landmarks of shape {} : {}".format(i, dataset.landmarks[i]), end="\n\n")

Size of shape 0 : torch.Size([518, 3])
Landmarks of shape 0 : tensor([[ 0.0156, -0.0755,  0.2701],
        [ 0.0190,  0.0418,  0.2677]])

Size of shape 1 : torch.Size([446, 3])
Landmarks of shape 1 : tensor([[0.0314, 0.0201, 0.2025],
        [0.0319, 0.0535, 0.1964]])

Size of shape 2 : torch.Size([465, 3])
Landmarks of shape 2 : tensor([[-0.0111,  0.0480,  0.1389],
        [-0.0108, -0.0148,  0.1165]])

Size of shape 3 : torch.Size([444, 3])
Landmarks of shape 3 : tensor([[ 0.0917, -0.3005,  0.1144],
        [ 0.0360,  0.2349,  0.0129]])

